<a href="https://colab.research.google.com/github/awaris123/OSNA-Project2/blob/main/GPUcolab_train_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install -U sentence-transformers
from sentence_transformers import SentenceTransformer


     |████████████████████████████████| 71kB 4.3MB/s 
     |████████████████████████████████| 1.3MB 7.5MB/s 
     |████████████████████████████████| 2.9MB 29.9MB/s 
     |████████████████████████████████| 1.1MB 52.6MB/s 
     |████████████████████████████████| 890kB 50.5MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.9-cp36-none-any.whl size=101036 sha256=53289770ea5fe7f848c19cb69010f45f402b6a11f2b143b3626b24031d65fd5a
  Stored in directory: /root/.cache/pip/wheels/fc/89/43/f2f5bc00b03ef9724b0f6254a97eaf159a4c4ddc024b33e07a
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=bc5ba72fa8e24843a0e8a27e1f26289305cdd9832a579a565060762d487cd77f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [1]:
import numpy as np
import keras
import pandas as pd
import pdb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [ ]:
#    def __init__(self, df, y_data, dim=(768*2), batch_size=200, n_classes=3, shuffle=True):
from simple_classifier_model import SimpleClassifier
df=pd.read_csv("https://raw.githubusercontent.com/awaris123/OSNA-Project2/main/option1-data/train.csv")
label_encoder = LabelEncoder()
targets = label_encoder.fit_transform(df['label'].values)

X_train, X_test, y_train, y_test = train_test_split(df[['title1_en', 'title2_en']], targets, test_size=0.2)
df_train = pd.DataFrame()

params={'lr': 0.01, 'opt': 'adam', 'model': 'SimpleClassifier', 'loss': 'crossentropy', 'batch_size': 25000}



ModuleNotFoundError: ignored

In [ ]:
from data_generator import DataGenerator
import pickle

training_generator = DataGenerator(X_train, y_train, total_samples=y_train.shape[0], batch_size=params['batch_size'])
validation_generator = DataGenerator(X_test, y_test, total_samples=y_test.shape[0], batch_size=params['batch_size'])
all_generator = DataGenerator(df, targets, total_samples=targets.shape[0], batch_size=params['batch_size'])
joint_embedding = np.empty((all_generator.total_samples, *all_generator.dim))
y = np.empty((all_generator.total_samples), dtype=int)

for i in range(int(all_generator.total_samples/all_generator.batch_size)):
  #joint_embedding[i*all_generator.batch_size:(i+1)*all_generator.batch_size,:], y[i*all_generator.batch_size:(i+1)*all_generator.batch_size]=all_generator.__getitem__(i) #
  e1, y1=all_generator.__getitem__(i) #

  with open(f"./joint_embedding_{i}", 'wb') as f:
    pickle.dump(e1, f)

  with open(f"./y_{i}", 'wb') as f:
    pickle.dump(y1, f)

0 / 256442
25000 / 256442
50000 / 256442
75000 / 256442
100000 / 256442
125000 / 256442
150000 / 256442
175000 / 256442
200000 / 256442
225000 / 256442


In [ ]:
del df
del targets
import pickle
with open("./joint_embedding", 'wb') as f:
  pickle.dump(joint_embedding, f)

with open("./y", 'wb') as f:
  pickle.dump(y, f)

NameError: ignored

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
mkdir ./drive/MyDrive/joint_embeddings


In [ ]:
!cp -r joint_embedding ./drive/MyDrive/joint_embeddings/
!cp -r y ./drive/MyDrive/joint_embeddings/

In [5]:
import pickle
path="./drive/MyDrive/joint_embeddings/"
batch_size=25000
X_all = np.empty((batch_size*10, 1536))
y_all = np.empty((batch_size*10))


for i in range(10):
  with open(f"{path}/joint_embedding_{i}", 'rb') as f:
    X=pickle.load(f)
    X_all[i*batch_size:(i+1)*batch_size,:]=X

  with open(f"{path}/y_{i}", 'rb') as f:
    y=pickle.load(f)
    y_all[i*batch_size:(i+1)*batch_size]=y
  print(i)



0
1
2
3
4
5


KeyboardInterrupt: ignored

In [6]:
with open("./joint_embedding", 'wb') as f:
  pickle.dump(X_all, f)

with open("./y", 'wb') as f:
  pickle.dump(y_all, f)

KeyboardInterrupt: ignored

In [7]:
#load
path="./drive/MyDrive/joint_embeddings/"

with open(f"{path}joint_embedding", 'rb') as f:
  X_all=pickle.load(f)

with open(f"{path}y", 'rb') as f:
  y_all=pickle.load(f)

In [4]:
!cp /content/drive/MyDrive/joint_embeddings/data_generator.py .
!cp /content/drive/MyDrive/joint_embeddings/simple_classifier_model.py .

In [8]:
#train from preprocessed data
from data_generator import DataGenerator
import pickle

params={'lr': 0.01, 'opt': 'adam', 'model': 'SimpleClassifier', 'loss': 'crossentropy', 'batch_size': 128}

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2)
del X_all
del y_all
training_generator = DataGenerator(X_train, y_train, total_samples=y_train.shape[0], batch_size=params['batch_size'])
validation_generator = DataGenerator(X_test, y_test, total_samples=y_test.shape[0], batch_size=params['batch_size'])

In [ ]:
import simple_classifier_model #import SimpleClassifier
import importlib
importlib.reload(simple_classifier_model)

model = simple_classifier_model.SimpleClassifier(input_shape=768*2)
model.model.summary()

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=params['lr'],
    decay_steps=10000,
    decay_rate=0.9)
opt = keras.optimizers.Adam(learning_rate=lr_schedule)

model.model.compile(optimizer=opt, loss=keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])#, keras.metrics.Precision(), keras.metrics.Recall()])
model.model.fit_generator(generator=training_generator,
                          validation_data=validation_generator,
                          epochs=100,
                          workers=2,
                    use_multiprocessing=False,
                    verbose=1)
model.model.save("/content/drive/MyDrive/joint_embeddings/test/bert_mlp_unbalanced_1500_bs128")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1500)              2305500   
_________________________________________________________________
batch_normalization (BatchNo (None, 1500)              6000      
_________________________________________________________________
dropout (Dropout)            (None, 1500)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               375250    
_________________________________________________________________
batch_normalization_1 (Batch (None, 250)               1000      
_________________________________________________________________
dropout_1 (Dropout)          (None, 250)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                1

In [ ]:
del X_all
del y_all


In [ ]:
# EVALUATION
#model=keras.models.load("/content/drive/MyDrive/joint_embeddings/test/bert_mlp_unbalanced_1500")
df=pd.read_csv("https://raw.githubusercontent.com/awaris123/OSNA-Project2/main/option1-data/train.csv")
encoder = LabelEncoder()
encoder.fit_transform(df['label'].values)

df=pd.read_csv("/content/drive/MyDrive/joint_embeddings/test.csv", sep=',')
final_df=pd.DataFrame(data=None, columns=['id', 'label'])
final_df['id'] = df['id']
targets=np.empty((final_df.shape[0]), dtype=int)
all_generator = DataGenerator(df, targets, total_samples=targets.shape[0], batch_size=128, pretrained_embeddings=False)
print(int(all_generator.total_samples/all_generator.batch_size))
for i in range(int(all_generator.total_samples/all_generator.batch_size)):
  #joint_embedding[i*all_generator.batch_size:(i+1)*all_generator.batch_size,:], y[i*all_generator.batch_size:(i+1)*all_generator.batch_size]=all_generator.__getitem__(i) #
  e1, _=all_generator.__getitem__(i) #
  targets[i*all_generator.batch_size:(i+1)*all_generator.batch_size]=np.argmax(model.model.predict(e1), axis=1)
  print(i, end = ' ')

#label decoder
final_df['label'] = encoder.inverse_transform(targets)

#convert to csv
final_df.to_csv("/content/drive/MyDrive/joint_embeddings/sample_submission.csv", index=False)

500
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 27

In [ ]:
print("test")

test
